In [18]:
%pip install tensorflow[and-cuda]==2.15.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
!export CUDNN_PATH=$(dirname $(python3 -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))
!export LD_LIBRARY_PATH=${CUDNN_PATH}/lib

!echo $CUDNN_PATH
!ls $CUDNN_PATH


 20180596_Yazan-Matarweh.png   haarcascade_frontalface_default.xml
 An.jpg			       indy2.jpg
 emb_den_wei.pkl	       indy.jpg
 file_modelatm2.h5	       R_O_D.jpg
 file_modelatm2.keras	       TEMP_model_test.ipynb
 file_modelatm3.keras	       TEMP_model_test.py
 file_model.h5		       WIN_20240517_22_16_08_Pro.jpg
'GradProject[1].ipynb'


In [20]:
import tensorrt
import cv2, numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
import os
from tensorflow.keras.layers import  Conv2D, Dense, MaxPooling2D, Input, Flatten, Layer
# from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model,Model


from time import sleep


In [21]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print(gpu_devices)
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    ## Magic happens here - similarity calculation
    def call(self, inputs):
        input_embedding, validation_embedding = inputs
        return tf.math.abs(input_embedding - validation_embedding)
    



In [23]:
# model = load_model("file_modelatm3.keras",custom_objects={'L1Dist':L1Dist})
# model.training = False

# for layer in model.layers: 
#     print(layer.get_config(), layer.get_weights())
    
#     # print(layer.get_weights())

# model.summary()

In [24]:
# lays = model.layers

# lays_dict = dict(  [ (lay.name,lay) for lay in lays]     )

# print(lays_dict['input_img'],lays_dict['validation_img'],sep='\n')

# embedding_weights = lays_dict['Embedding_Layer'].get_weights()
# dense_weights, dense_biases = lays_dict['dense_1'].get_weights()

In [25]:
def Embedding_Layer(): 
    input_layer = Input(shape=(100,100,3), name='input_image')
    
    #first block
    convolution_layer1 = Conv2D(64, (10,10), activation='relu')(input_layer)
    max_pooling1 = MaxPooling2D(64, (2,2), padding='same')(convolution_layer1)
    
    #second block
    convolution_layer2 = Conv2D(128, (7,7), activation='relu')(max_pooling1)
    max_pooling2 = MaxPooling2D(64, (2,2), padding='same')(convolution_layer2)
    
    #third block 
    convolution_layer3 = Conv2D(128, (4,4), activation='relu')(max_pooling2)
    max_pooling3 = MaxPooling2D(64, (2,2), padding='same')(convolution_layer3)
    
    #fourth & final block
    convolution_layer4 = Conv2D(256, (4,4), activation='relu')(max_pooling3)
    flatten_layer1 = Flatten()(convolution_layer4)
    dense_layer1 = Dense(4096, activation='sigmoid')(flatten_layer1)
    
    
    return Model(inputs=input_layer, outputs=dense_layer1, name='Embedding_Layer')
    
model1= Embedding_Layer()

def siamese_model(): 
    
    # base image input into the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siameselayer = L1Dist(name='l1_layer')
    dist = siameselayer([model1(input_image), model1(validation_image)])
    
    # Classification layer 
    classify = Dense(1, activation='sigmoid',name='output_layer')(dist)
    
    return Model(inputs=[input_image, validation_image], outputs=classify, name='SiameseNetwork')


SiameseModel = siamese_model()
# SiameseModel.get_layer('Embedding_Layer').set_weights(embedding_weights)
# SiameseModel.get_layer('output_layer').set_weights((dense_weights,dense_biases))

# for layer in SiameseModel.layers: 
#     print(layer.get_config(), layer.get_weights())


In [26]:
SiameseModel.summary()


Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 Embedding_Layer (Functiona  (None, 4096)                 3896044   ['input_img[0][0]',           
 l)                                                       8          'validation_img[0][0]']      
                                                                                     

In [27]:
# for layer in model.layers: 
#     print(layer.get_config(), layer.get_weights())


In [28]:
print(SiameseModel.get_layer('Embedding_Layer').get_weights() )


[array([[[[ 6.98600337e-03, -2.05809064e-02, -1.17977746e-02, ...,
           1.01542808e-02, -9.83431563e-03,  6.53264299e-03],
         [-2.29009036e-02,  2.68317424e-02, -8.17087851e-03, ...,
          -6.65984675e-03, -2.55544297e-02, -2.08247155e-02],
         [ 1.46650411e-02,  2.41576470e-02, -2.39633601e-02, ...,
           8.82263854e-03, -2.35731322e-02,  1.97181031e-02]],

        [[ 8.98217782e-03, -1.93538666e-02,  1.82826445e-03, ...,
          -1.93328839e-02,  6.08523563e-03,  2.92049348e-03],
         [ 2.18942128e-02, -8.43810290e-03,  2.50278190e-02, ...,
          -1.58715416e-02, -2.84666941e-03, -7.91628286e-03],
         [-7.49876350e-03, -2.26924755e-03,  1.22598484e-02, ...,
           2.14343220e-02, -2.36636437e-02, -1.37206744e-02]],

        [[ 1.30368732e-02, -1.47649078e-02,  2.80717500e-02, ...,
          -2.23570298e-02,  1.86123662e-02, -9.70007479e-03],
         [ 1.74263678e-02, -1.89566985e-02,  2.63800696e-02, ...,
          -1.64703466e-03,  6.165

In [29]:
import pickle
embedding_weights, dense_weights = pickle.load(open('emb_den_wei.pkl','rb'))
SiameseModel.get_layer('Embedding_Layer').set_weights(embedding_weights)
SiameseModel.get_layer('output_layer').set_weights(dense_weights)

In [30]:
# SiameseModel.save(filepath='./file_modelatm3.keras')

In [31]:
model = load_model("file_modelatm3.keras",custom_objects={'L1Dist':L1Dist})

In [32]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 Embedding_Layer (Functiona  (None, 4096)                 3896044   ['input_img[0][0]',           
 l)                                                       8          'validation_img[0][0]']      
                                                                                     

In [33]:
def preprocessing_func(file_path):
    
    # Read in image from file path
    byte_image = tf.io.read_file(file_path)
    # Load image 
    imag = tf.io.decode_jpeg(byte_image)
    
    # Preprocessing: resize image -> 100*100*3
    imag = tf.image.resize(imag, (100,100))
    # Scale image to be between 0 and 1 
    imag = imag / 255.0
    plt.figure()
    plt.imshow(imag)
    imag = np.expand_dims(imag,axis=0)
    return imag


# def preprocess_identical(input_img, validation_img, label):
#     #label: indicates whether the input and validation images are identical or not
#     return(preprocessing_func(input_img), preprocessing_func(validation_img), label)

def get_face(file_path):
    # byte_image = tf.io.read_file(file_path)
    # img = tf.io.decode_jpeg(byte_image) if '.jpg' in file_path or '.jpeg' in file_path else tf.io.decode_png(byte_image)

    img = cv2.imread(file_path)
    face_classifier = cv2.CascadeClassifier(os.path.join(cv2.data.haarcascades, "haarcascade_frontalface_default.xml"))
    
    classifier = face_classifier.detectMultiScale( 
    img, scaleFactor=1.1, minNeighbors=9) 

    x0,y0,w,h = classifier[0]
    
    x1 = x0 + w
    y1 = y0 + h
    face = img[y0:y1,x0:x1]
    face = cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
    face = face / 255.0
    # plt.figure()
    # plt.imshow(face)
    face = tf.image.resize(face,(100,100))
    return np.expand_dims(face, axis=0)

def get_faces_identical(input_img, validation_img, label):
#     #label: indicates whether the input and validation images are identical or not
    return(get_face(input_img), get_face(validation_img), label)



In [34]:
filename_known1 = '../Known_Faces/20180596_Yazan-Matarweh.png'
filename_known2 = '../Known_Faces/20021736_Anne-Hathaway.png'
filename_known3 = '../Known_Faces/19850192_Rosie-O\'Donnell.jpg'

filename_unknown1 = "WIN_20240517_22_16_08_Pro.jpg"
filename_unknown2 = "R_O_D.jpg"
filename_unknown3 = 'An.jpg'


arenas = (
    ([filename_known1,filename_unknown1],[filename_known2,filename_unknown1],[filename_known3,filename_unknown1]),
    ([filename_known1,filename_unknown2],[filename_known2,filename_unknown2],[filename_known3,filename_unknown2]),
    ([filename_known1,filename_unknown3],[filename_known2,filename_unknown3],[filename_known3,filename_unknown3])

)



for challenges in arenas:
    for challenge in challenges:
        challenge = [get_face(im) for im in challenge]
        prediction = SiameseModel.predict(challenge)
        print(prediction[0])        
    # print('\n'); plt.figure(); plt.imshow(np.ones((100,100)))




1/1 [==============================] - 0s 143ms/step
[0.95348334]
1/1 [==============================] - 0s 18ms/step
[0.99996245]
1/1 [==============================] - 0s 21ms/step
[0.6819929]
1/1 [==============================] - 0s 19ms/step
[0.8464312]
1/1 [==============================] - 0s 18ms/step
[0.07115254]
1/1 [==============================] - 0s 19ms/step
[0.998002]
1/1 [==============================] - 0s 19ms/step
[0.9258282]
1/1 [==============================] - 0s 18ms/step
[0.91372603]
1/1 [==============================] - 0s 19ms/step
[0.8823038]
